# Launch CB-LLM

---
Goal of the notebook: .

Inputs of the notebook:
-
Output of the notebook:
-
Takeaways: 
- .
- .

In [ ]:
# !pip install -r requirements.txt

In [ ]:
import sys
sys.path.append('../../run_experiments/')
sys.path.append('../../run_experiments/scripts')
sys.path.append('../../run_experiments/models')
sys.path.append('../../run_experiments/data')


In [ ]:

from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, Dataset

import pickle 

# import fonction for getting PLM and tokenizer
from models.utils import load_model_and_tokenizer


# library for managing memory RAM
import gc

In [ ]:
torch.cuda.empty_cache()
gc.collect()

# 1.SETUP ENVIRONMENT VARIABLES

In [ ]:
# import config
from load_config import load_config

model_name = 'bert-base-uncased'    # 'bert-base-uncased' ou 'deberta-large' or 'gemma'
dataset    = 'movies'               # 'movies' / 'agnews' / 'dbpedia' / 'medical'/ 'ledgar'/ n24news
annotation = 'C3M'       # 'C3M' ou 'our_annotation' ou 'combined_annotation'
config = load_config(model_name, dataset)
config.annotation = annotation

In [ ]:
model, tokenizer, _ , _ = load_model_and_tokenizer(config, n_concepts = 1)

# 2. Data Loading and Preparation

In [ ]:

# Prepare your dataset (replace with your actual dataset)
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, concepts, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.concepts = concepts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        if self.concepts is not None:
            concept = self.concepts[idx]
            encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'concepts': torch.tensor(concept, dtype=torch.float),
                'label': torch.tensor(label, dtype=torch.long)
            }
        else :
            encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'label': torch.tensor(label, dtype=torch.long)
            }    

In [ ]:
df_aug_train = pd.read_csv(f"{config.SAVE_PATH_CONCEPTS}/df_with_topics_v4.csv")
df_aug_test = pd.read_csv(f"{config.SAVE_PATH_CONCEPTS}/df_with_topics_v4_test.csv")
df_aug_train_cbllm_acc = pd.read_csv(f"{config.SAVE_PATH_CONCEPTS}/df_with_topics_v4_CB_LLM_ACC.csv")

columns = [c for c in df_aug_train_cbllm_acc.columns if "dummy" not in c and c not in ["Unnamed: 0.1", "Unnamed: 0", "test"]]
df_aug_train_cbllm_acc = df_aug_train_cbllm_acc[columns]
concept_col = df_aug_train_cbllm_acc.columns[2:]
train_concepts = df_aug_train_cbllm_acc[concept_col].values.tolist()

In [ ]:
train_dataset = TextClassificationDataset(df_aug_train_cbllm_acc.text, df_aug_train_cbllm_acc.label, train_concepts, tokenizer, max_length=128)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = TextClassificationDataset(df_aug_test.text, df_aug_test.label, train_concepts, tokenizer, max_length=128)
test_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
#Entrainemet concept seulement

code below used for xp on bert without residual part

In [ ]:
class CBL(nn.Module):
    def __init__(self, base_model, tokenizer, concept_dim, max_length, l1_ratio=0.99, dropout = 0.1):
        super().__init__()
        import copy
        self.base_model = copy.deepcopy(base_model)
        for p in self.base_model.parameters():
            p.requires_grad = True
        self.tokenizer = tokenizer
        self.concept_extractor = nn.Linear(base_model.config.hidden_size, concept_dim)
        self.max_length = max_length
        self.gelu = nn.GELU()
        self.fc = nn.Linear(concept_dim, concept_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state[:, 0, :]
        concepts = F.relu(self.concept_extractor(last_hidden_state)) #here
        # concepts = F.relu(self.concept_extractor(last_hidden_state)) #here
        # concepts = self.gelu(self.concept_extractor(last_hidden_state)) #here
        concepts = self.fc(concepts) #here
        concepts = self.dropout(concepts) #here
        concepts = concepts + self.concept_extractor(last_hidden_state) #here
        return concepts
    
    def get_regularization(self):
        return self.classifier.regularization()

In [ ]:
#trouvé dans le code source dez CB_LLM
def cos_sim_cubed(cbl_features, target):
    cbl_features = cbl_features - torch.mean(cbl_features, dim=-1, keepdim=True)
    target = target - torch.mean(target, dim=-1, keepdim=True)

    cbl_features = F.normalize(cbl_features**3, dim=-1)
    target = F.normalize(target**3, dim=-1)

    sim = torch.sum(cbl_features*target, dim=-1)
    return sim.mean()

In [ ]:
concept_dim = df_aug_train_cbllm_acc.shape[1]-2

In [ ]:
train_dataset = TextClassificationDataset(df_aug_train_cbllm_acc.text, df_aug_train_cbllm_acc.label, train_concepts, tokenizer, max_length=128)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = TextClassificationDataset(df_aug_test.text, df_aug_test.label, train_concepts, tokenizer, max_length=128)
test_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:

# Initialisation de la meilleure loss à un très grand nombre
best_loss = float('inf')

# Hyperparameters
concept_dim = df_aug_train_cbllm_acc.shape[1]-2  # Number of concepts

# num_classes = df_aug_train_cbllm_acc.label.nunique()  # Number of classes

learning_rate = 0.0001

num_epochs = 5

max_length = 128

# Initialize model, loss functions, and optimizer
cbl = CBL(model, tokenizer, concept_dim, max_length)
cbl.to(config.device)

# concept_criterion = nn.MSELoss()
concept_criterion = cos_sim_cubed
optimizer = optim.Adam(cbl.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    cbl.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        
        input_ids = batch['input_ids'].to(config.device)
        attention_mask = batch['attention_mask'].to(config.device)
        true_concepts = batch['concepts'].to(config.device)
        
        optimizer.zero_grad()
        
        # Forward pass
        predicted_concepts = cbl(input_ids, attention_mask)

        # Compute losses
        concept_loss = concept_criterion(predicted_concepts, true_concepts)
        
        # total_loss = concept_loss 
        total_loss = -concept_loss 

        # print("classiff_loss", classification_loss)
        # print("-------------------")
        # print("concept_loss", concept_loss)
        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()

        del input_ids, attention_mask, true_concepts, predicted_concepts, 

    # Sauvegarde du meilleur modèle
    if total_loss.item() < best_loss:
        best_loss = total_loss.item()
        torch.save(cbl.state_dict(), f'{config.SAVE_PATH}/cbl_{config.model_name}_best_model_iteration.pth')
        print(f"New best model saved with loss: {best_loss:.4f}")
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss.item()}")

In [ ]:
torch.save(cbl.state_dict(), f'{config.SAVE_PATH}/cbl_{config.model_name}_last_model_iteration.pth')
print(f"Last model saved with loss: {total_loss:.4f}")

In [ ]:
# Re-créer l'instance du modèle
cbl = CBL(model, tokenizer, concept_dim, max_length)
cbl.load_state_dict(torch.load(f'{config.SAVE_PATH}/cbl_{config.model_name}_best_model_iteration.pth'))
cbl.to(config.device)
cbl.eval()  # Passage en mode évaluation si nécessaire

In [ ]:
import copy

In [ ]:
class ElasticNetLayer(nn.Module):
    def __init__(self, input_dim, output_dim, l1_ratio=0.99):
        super(ElasticNetLayer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.l1_ratio = l1_ratio

    def forward(self, x):
        return self.linear(x)

    def regularization(self):
        l1_reg = torch.norm(self.linear.weight, p=1)
        l2_reg = torch.norm(self.linear.weight, p=2)
        return self.l1_ratio * l1_reg + (1 - self.l1_ratio) * l2_reg

class CB_LLM(nn.Module):
    def __init__(self, cbl_model, tokenizer, concept_dim, num_classes, max_length, l1_ratio=0.99, dropout = 0.1):
        super().__init__()
        self.cbl_model = copy.deepcopy(cbl_model)
        self.tokenizer = tokenizer
        # self.concept_extractor = nn.Linear(base_model.config.hidden_size, concept_dim)
        self.max_length = max_length
        self.classifier = ElasticNetLayer(concept_dim, num_classes, l1_ratio)

        # Freeze parameters of cbl_model
        for param in self.cbl_model.parameters():
            param.requires_grad = False


    def forward(self, input_ids, attention_mask):
        concepts = self.cbl_model(input_ids=input_ids, attention_mask=attention_mask)
        output = self.classifier(F.relu(concepts))
        return concepts, output
    
    def get_regularization(self):
        return self.classifier.regularization()

In [ ]:
train_dataset = TextClassificationDataset(df_aug_train_cbllm_acc.text, df_aug_train_cbllm_acc.label, train_concepts, tokenizer, max_length=256)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = TextClassificationDataset(df_aug_test.text, df_aug_test.label, None, tokenizer, max_length=256)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=True)

In [ ]:
# Hyperparameters
concept_dim = df_aug_train_cbllm_acc.shape[1]-2  # Number of concepts
num_classes = df_aug_train_cbllm_acc.label.nunique()  # Number of classes
# learning_rate = 0.001
learning_rate = 0.001

num_epochs = 20
# lambda_concept = 1  # Weight for concept loss
lambda_concept = 0.05  # Weight for concept loss

max_length = 128

# Initialize model, loss functions, and optimizer
cb_llm_model = CB_LLM(cbl_model = cbl, tokenizer = tokenizer, concept_dim = concept_dim, num_classes = num_classes, max_length = max_length)
cb_llm_model.to(config.device)
# concept_criterion = nn.MSELoss()
classification_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cb_llm_model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    cb_llm_model.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        
        input_ids = batch['input_ids'].to(config.device)
        attention_mask = batch['attention_mask'].to(config.device)
        true_concepts = batch['concepts'].to(config.device)
        labels = batch['label'].to(config.device)
        
        optimizer.zero_grad()
        
        # Forward pass
        predicted_concepts, predicted_output = cb_llm_model(input_ids, attention_mask)
        
        # Compute losses
        # concept_loss = concept_criterion(predicted_concepts, true_concepts)
        classification_loss = classification_criterion(predicted_output, labels)
        regularization = cb_llm_model.get_regularization()
        
        # Joint loss
        # total_loss = classification_loss + lambda_concept * concept_loss +  0.0007 * regularization
        total_loss = classification_loss +  0.0007 * regularization
        # print("classiff_loss", classification_loss)
        # print("-------------------")
        # print("concept_loss", concept_loss)
        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()
    
    #Epoch eval
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss.item()}")
    cb_llm_model.eval()
    accuracy = 0.
    predict_labels = np.array([])
    true_labels = np.array([])
    for batch_eval in (test_dataloader):
        input_ids_eval = batch_eval["input_ids"].to(config.device)
        attention_mask_eval = batch_eval["attention_mask"].to(config.device)
        label_eval = batch_eval["label"].to(config.device)
        logits = cb_llm_model(input_ids=input_ids_eval, attention_mask=attention_mask_eval)[1]
        predictions = torch.argmax(logits, axis=1)
        accuracy += torch.sum(predictions == label_eval).item()
        predict_labels = np.append(predict_labels, predictions.cpu().numpy())
        true_labels = np.append(true_labels, label_eval.cpu().numpy())

        # Libérer la mémoire GPU après chaque batch
        del input_ids_eval, attention_mask_eval, label_eval, logits, predictions
        torch.cuda.empty_cache()

    accuracy /= len(test_dataloader.dataset)
    num_true_labels = len(np.unique(true_labels))
    macro_f1_scores = []
    for label in range(num_true_labels):
        label_pred = np.array(predict_labels) == label
        label_true = np.array(true_labels) == label
        macro_f1_scores.append(f1_score(label_true, label_pred, average='macro'))
    mean_macro_f1_score = np.mean(macro_f1_scores)
    print(f"Acc = {accuracy*100} Macro F1 = {mean_macro_f1_score*100}")



In [ ]:
print(f"Acc = {accuracy*100} Macro F1 = {mean_macro_f1_score*100}")


# save the model

In [ ]:
# save en pth cb_llm_model et cbl
torch.save(cbl.state_dict(), f'{config.SAVE_PATH}/cbl_{config.model_name}.pth')
torch.save(cb_llm_model.state_dict(), f'{config.SAVE_PATH}/cb_llm_model_{config.model_name}.pth')


# Evaluation of the model

In [ ]:
# Supposons que sont déjà importées ou définies nos classes : CBL et CB_LLM

# Hyperparameters
concept_dim = df_aug_train_cbllm_acc.shape[1]-2  # Number of concepts

# num_classes = df_aug_train_cbllm_acc.label.nunique()  # Number of classes

learning_rate = 0.0001

num_epochs = 4

max_length = 512 # ne sert à rien en vrai

# Initialize model, loss functions, and optimizer
cbl = CBL(model, tokenizer, concept_dim, max_length)
cbl.to(config.device)

# Chargement des poids sauvegardés pour 'cbl'
cbl_state = torch.load(f'{config.SAVE_PATH}/cb_llm_checkpoints/cbl_{config.model_name}.pth', map_location=config.device)    
cbl.load_state_dict(cbl_state)
cbl.to(config.device)
cbl.eval()

print("Modèles chargés et prêts pour l'évaluation.")


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Supposons que vous ayez récupéré les noms des concepts depuis votre DataFrame
# Par exemple, si les deux premières colonnes ne sont pas des concepts :
concept_names = list(df_aug_train_cbllm_acc.columns[2:])  # Liste des noms de concepts

##############################################
# 1. Calculer la médiane sur les valeurs réelles du train
##############################################
true_concepts_train_all = []

with torch.no_grad():
    for batch in train_dataloader:
        # On récupère les valeurs réelles des concepts
        true_concepts = batch["concepts"].to(config.device)  # Forme attendue : [batch_size, num_concepts]
        true_concepts_train_all.append(true_concepts.cpu().numpy())

# Concaténation sur toutes les batchs
true_concepts_train_all = np.concatenate(true_concepts_train_all, axis=0)  # Forme : (N_train, num_concepts)

# Calcul de la médiane pour chaque concept (le long de l'axe 0)
median_per_concept = np.median(true_concepts_train_all, axis=0)

print("Médianes calculées sur le jeu d'entraînement :")
for i, median_val in enumerate(median_per_concept):
    concept_name = concept_names[i] if i < len(concept_names) else f"Concept {i}"
    print(f"  {concept_name}: {median_val:.3f}")

##############################################
# 2. Évaluation sur un autre jeu (par exemple, le test)
##############################################
predicted_concepts_all = []
true_concepts_all = []

cbl.eval()
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(config.device)
        attention_mask = batch["attention_mask"].to(config.device)
        true_concepts = batch["concepts"].to(config.device)  # Taille : [batch_size, num_concepts]
        
        # Obtenir les concepts prédits (le second output est ignoré ici)
        predicted_concepts = cbl(input_ids, attention_mask)

        # Déplacement sur CPU et conversion en numpy
        predicted_concepts_all.append(predicted_concepts.cpu().numpy())
        true_concepts_all.append(true_concepts.cpu().numpy())

predicted_concepts_all = np.concatenate(predicted_concepts_all, axis=0)  # Forme : (N_test, num_concepts)
true_concepts_all = np.concatenate(true_concepts_all, axis=0)            # Forme : (N_test, num_concepts)

num_concepts = predicted_concepts_all.shape[1]
f1_scores = []

# Pour chaque concept, on binarise avec la médiane calculée sur le train
for i in range(num_concepts):
    median_value = median_per_concept[i]  # Seuil défini par le train pour le concept i
    
    # Récupérer les valeurs pour le concept i sur l'ensemble d'évaluation
    pred_values = predicted_concepts_all[:, i]
    true_values = true_concepts_all[:, i]
    
    # Binarisation avec le même seuil (la médiane du train)
    pred_binary = (pred_values > median_value).astype(int)
    true_binary = (true_values > median_value).astype(int)
    
    # Calcul du F1-score
    f1 = f1_score(true_binary, pred_binary)
    f1_scores.append(f1)
    
    concept_name = concept_names[i] if i < len(concept_names) else f"Concept {i}"
    print(f"Concept '{concept_name}': Médiane (train) = {median_value:.3f}, F1 score = {f1:.3f}")

mean_f1 = np.mean(f1_scores)
print("F1 score moyen sur tous les concepts :", mean_f1)
